<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/%E6%83%85%E5%A0%B1%E8%AA%8D%E8%AD%98%E8%AB%962021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 情報認識論試験問題

以下の問題について解答し、結果を提出してください。なお、解答については文章はテキストセルに、プログラムに関してはコードセルに記入すること。また、結果の提出については、この画面の左上の「ファイル」メニュー内にある「ダウンロード」->「.ipynbをダウンロード」で自分のマシンにダウンロードできるので、それを各自の学内メールに添付してください。

# 1. ニューラルネット

(1) ニューラルネットを学習させる際に用いられる「誤差逆伝播法」について説明せよ。

【解答欄】

(2) 誤差逆伝播法では、微分操作が必要となる。以下のネットワークを考える。

- 入力層：$x = (x_1, x_2, ..., x_i, ..., x_n)$
- 第１層：$y = x + 2$
- 第２層：$z = y \odot y \times 3$ (注：$\odot$ はアダマール積（成分ごとの積）)
- 出力層：$out = mean(z)$ （$z_i$の平均値）

このとき、出力値に対する入力値の微分
$\frac{\partial}{\partial x_{i}}out$ を計算せよ。

【解答欄】
$$ \frac{\partial}{\partial x_{i}}out = ここに数式を書く $$

(3) $x = (1, 2, 3, 4)$ となった時の、$\frac{\partial}{\partial x}out$ を求めよ。





【解答欄】
$$
(\frac{\partial}{\partial x_1}out, \frac{\partial}{\partial x_2}out, \frac{\partial}{\partial x_3}out, \frac{\partial}{\partial x_4}out) = ここに数値を書く
$$

# 2. Pytorch

(1) 本講義では、ニューラルネットの実装に Pytorch を用いたが、そこで用いられる基本的なデータ構造に「テンソル」がある。テンソルとはどのようなもので、ニューラルネットにおいてはどのように使われるかを説明せよ。

【解答欄】
ここに記述

(2) 2 x 2 のリスト [[1, 2], [3, 4]] からテンソルを作り、x という変数に代入せよ。

（ヒント：テンソルの作成には、torch.Tensor() を用いる）

In [ ]:
import torch

l =  # ここにリストを書く
print("list: ", l)

x =  # ここでテンソルに変換するコードを書く
print("x: ", x)

Pytorch には微分を自動で行なってくれるため、上記の誤差逆伝播法が簡単に実行できる。そのためには、テンソルの requires_grad 属性を True にしておく必要がある。


In [ ]:
x.requires_grad = True # これはそのまま実行してください。

(3) 問１でのネットワークをテンソル x を用いて構築せよ。

In [ ]:
y = # ここを自分で埋める
z = # ここも自分で埋める埋める
out = z.mean() # ここは (z1+z2+z3+z4)/4 でもいいですが、平均なので専用関数があります。

(4) backword()関数を用いて、out に対する x の微分値を求めよ。

In [ ]:
out.backward() # これで逆伝播を行う
x.grad # x の grad 属性に微分値が格納される

# 3. Pytorch を用いた実例

ラーメンは今や世界各国で食べられています。ここでは、メーカー（Brand）, 形態＝袋麺/カップ麺（Style）, 生産国（Country）から星の数＝美味しさ（Stars）を予測します。

注：前置きが長いです。問題は一番下にあります。

In [ ]:
# ライブラリのインポート
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
 
# PyTorchのインポート
import torch 
import torch.nn as nn
 
# 評価指標（Scikit-learn）
from sklearn.metrics import mean_squared_error

### データの取得

元ネタ：https://www.kaggle.com/residentmario/ramen-ratings

In [ ]:
!curl https://raw.githubusercontent.com/ailab-nda/NLP/main/ramen-ratings.csv > ramen-ratings.csv

ダウンロードしたデータの読み込み

In [ ]:
ramen = pd.read_csv('ramen-ratings.csv')

### データの確認
（最初の5行を表示）

In [ ]:
ramen.head()

データのサイズ確認

In [ ]:
ramen.shape

ラーメンの国別登録数の確認

In [ ]:
ramen['Country'].value_counts()[0:10]

未評価（Unrated）のラーメンを除外

In [ ]:
mask = ramen.index[ramen['Stars'] == 'Unrated']
ramen = ramen.drop(index = mask)
ramen = ramen.reset_index(drop=True)
ramen.shape

Stars の欄が文字になっていので、数値に変換します。

In [ ]:
# Starsのデータ型を確認
print(ramen['Stars'].dtype)
 
# float型へ変換
ramen['Stars'] = ramen['Stars'].astype(float)
 
# 改めてデータ型を確認
print(ramen['Stars'].dtype)

不要な列（レビュー番号、商品名、トップ１０、は使いません）の削除

In [ ]:
ramen = ramen.drop(columns=['Review #', 'Top Ten', 'Variety'])
ramen.head()

特徴量のダミー変数化（カテゴリ変数を０、１で表現します。具体的には下の実行例を見てください）。

In [ ]:
Country = pd.get_dummies(ramen['Country'], prefix='Country', drop_first=True)
Brand = pd.get_dummies(ramen['Brand'], prefix='Brand',drop_first=True)
Style = pd.get_dummies(ramen['Style'], prefix='Style',drop_first=True)
 
# ダミー変数化した特徴量を結合
ramendf = pd.concat([Country, Brand,Style], axis=1)
 
# 確認
ramendf.head()

データを特徴量（X:ベクトル）と目的変数(y: スカラー）へ分割

In [ ]:
X = np.array(ramendf, dtype=np.float32) 
y = np.array(ramen[['Stars']], dtype=np.float32)

## モデルの作成

In [ ]:
# モデル
model = nn.Linear(397, 1)

# 損失関数
loss = nn.MSELoss()
 
# 最適化関数
optimizer = torch.optim.SGD(model.parameters(), lr=0.6)

モデルの学習

In [ ]:
for epoch in range(1000):
    # ステージ1 Numpy配列からテンソルへ変換
    inputs = torch.from_numpy(X)
    targets = torch.from_numpy(y)

    # ステージ2 推測値を出力して誤差（コスト）を算出
    outputs = model(inputs)
    cost = loss(outputs, targets)
    # ステージ3 誤差逆伝播（バックプロパゲーション）
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    # ステージ4 50回毎にコストを表示
    if (epoch+1) % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 1000, cost.item()))

予測値の出力

In [ ]:
y_pred = model(torch.from_numpy(X)).data.numpy()

#最初の５つを確認
print(y_pred[0:5]) #予測値
print(y[0:5]) #本当の値

予測の平均二乗誤差を確認

In [ ]:
mean_squared_error(y, y_pred)

### 問題
「日清が、タイで販売する、袋麺」の星の数はどれくらいと予想されるか？

In [ ]:
## ここで解答プログラムを実行